In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import os

# File path to your dataset
file_path = r"fulldataset.xlsx"
# Load the dataset
data = pd.read_excel(file_path, engine='openpyxl', dtype=str)

## **Data Preparation**

# Convert all column names to lowercase and remove spaces
data.columns = data.columns.str.strip().str.lower()

# Define the selected features based on relevance
selected_features = [
    'temp', 'nasal', 'eye', 'ears', 'cough', 'weight', 'total_steps', 'total_mi',
    'lying_bouts', 'lying_daily', 'milk_intake', 'milk_percent', 'starter_intake',
    'speed', 'speed_percent', 'brix', 'fecal', 'ultrasound'
]

# Target variable
target = 'brd_total'

# Check for missing columns and remove them
missing_cols = [col for col in selected_features if col not in data.columns]
if missing_cols:
    print(f"Warning: Missing columns in the dataset: {missing_cols}")
    selected_features = [col for col in selected_features if col in data.columns]

# Fill missing values with 0 or a more appropriate value
data[selected_features] = data[selected_features].fillna(0)  # Use 0 or another suitable value

# Collect features and target variable
X = data[selected_features]
y = data[target]

# Convert non-numeric columns to numeric
for col in X.columns:
    if X[col].dtype == 'object':  # Check if the column is of object type
        X[col] = pd.to_numeric(X[col], errors='coerce').fillna(0)  # Convert to numeric, replace errors with NaN, and then fill NaNs with 0

# Fill any remaining NaN values in X with 0
X = X.fillna(0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert target values to numeric and handle missing values
y_train = pd.to_numeric(y_train, errors='coerce').fillna(0)
y_test = pd.to_numeric(y_test, errors='coerce').fillna(0)

# Ensure target values start from 0 and are consecutive
y_train_unique = y_train.unique()
target_mapping = {val: i for i, val in enumerate(y_train_unique)}
y_train = y_train.map(target_mapping).astype(int)  # Ensure integer type
y_test = y_test.map(target_mapping).astype(int)  # Ensure integer type

# Standardize numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)  # Use .values to access the NumPy array
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)  # Use .values to access the NumPy array

## **Define Deep Q Knowledge Distillation Model**
class DQKD(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQKD, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Model Initialization
model = DQKD(input_dim=X_train_tensor.shape[1], output_dim=len(y_train_unique))

## **Loss function and Optimizer**
learning_rate = 0.001  # Define learning rate
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

## **Training the model**
def train_model(model, X_train, y_train, epochs=50):
    loss_values = []  # List to capture loss values at each epoch
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        
        loss_values.append(loss.item())  # Append the loss value to the list
        
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')
    
    return loss_values  # Return the list of loss values

loss_values = train_model(model, X_train_tensor, y_train_tensor)

## **Evaluating Model Performance**
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred_labels = torch.argmax(y_pred, axis=1)
    accuracy = (y_pred_labels == y_test_tensor).sum().item() / y_test_tensor.size(0)
    print(f'Model Accuracy: {accuracy * 100:.2f}%')

# **Tabulate Results**
# Create a DataFrame to display the loss values per epoch and the final accuracy
results = {
    'Epoch': list(range(50)),
    'Loss': loss_values + [None] * (50 - len(loss_values))  # Pad loss values to 50 epochs if less
}

# Adding accuracy at the end of the table
results_df = pd.DataFrame(results)
results_df['Accuracy'] = [None] * len(results_df)
results_df.loc[49, 'Accuracy'] = accuracy * 100  # Insert accuracy in the last row

# Adding learning rate and training rate to the table
results_df['Learning Rate'] = learning_rate
results_df['Training Rate (Epochs)'] = 50  # Training rate is the number of epochs

# Display the tabulated results
print("\nTraining Loss and Accuracy Table:")
print(results_df)


C:\Users\HP DRAGON FLY\AppData\Local\Temp\ipykernel_22468\255736262.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = pd.to_numeric(X[col], errors='coerce').fillna(0)  # Convert to numeric, replace errors with NaN, and then fill NaNs with 0


Epoch 0, Loss: 2.5758211612701416
Epoch 10, Loss: 2.38887095451355
Epoch 20, Loss: 2.16355037689209
Epoch 30, Loss: 1.9588457345962524
Epoch 40, Loss: 1.780250072479248
Model Accuracy: 34.59%

Training Loss and Accuracy Table:
    Epoch      Loss   Accuracy  Learning Rate  Training Rate (Epochs)
0       0  2.575821       None          0.001                      50
1       1  2.557789       None          0.001                      50
2       2  2.540076       None          0.001                      50
3       3  2.522427       None          0.001                      50
4       4  2.504715       None          0.001                      50
5       5  2.486748       None          0.001                      50
6       6  2.468320       None          0.001                      50
7       7  2.449384       None          0.001                      50
8       8  2.429842       None          0.001                      50
9       9  2.409660       None          0.001                      50
10 